In [1]:
from statsmodels.tsa.stattools import grangercausalitytests
import pandas as pd

from scipy.stats import chi2

In [2]:
WDI_df = pd.read_excel('WB_stock_market_gdp_data.xlsx')

In [3]:
WDI_df_first_dif = pd.concat([WDI_df.T.iloc[:4], WDI_df.T.iloc[4:].diff(periods=1)[1:]]).T
WDI_df_change = pd.concat([WDI_df.T.iloc[:4], WDI_df.T.iloc[4:].div(WDI_df.T.iloc[4:].shift(1)).iloc[1:]-1]).T

In [4]:
WDI_df = WDI_df_first_dif.copy()

In [5]:
WDI_df_hist = WDI_df.copy()
WDI_df_hist['name'] = WDI_df_hist['Country Code'] + ' ' +  WDI_df_hist['Indicator Code']
WDI_df_hist = WDI_df_hist.T

WDI_df_hist.columns = WDI_df_hist.iloc[-1]
WDI_df_hist = WDI_df_hist[4:-1]

In [6]:
WDI_df = pd.melt(WDI_df, id_vars=WDI_df.columns[:4], value_vars=WDI_df.columns[4:])
WDI_df = WDI_df.dropna(subset=['value'])

# Current LCU

In [7]:
WDI_df_marcap = WDI_df[WDI_df['Indicator Code'].isin(['CM.MKT.LCAP.CN'])].reset_index(drop=True)
WDI_df_gdp = WDI_df[WDI_df['Indicator Code'].isin(['NY.GDP.MKTP.CN'])].reset_index(drop=True)

lagvar = 8

## marcap is granger caused by gdp

In [8]:
ssr_based_chi2_marcap_by_gdp_cn = pd.DataFrame()

for country in WDI_df['Country Name'].unique():
    WDI_df_gdp_test = WDI_df_gdp[WDI_df_gdp['Country Name'].isin([country])].reset_index(drop=True)
    WDI_df_gdp_test = WDI_df_gdp_test.rename(columns={"value": WDI_df_gdp_test['Indicator Name'][0], "variable": 'Year'})

    WDI_df_gdp_test = WDI_df_gdp_test[['Year', WDI_df_gdp_test['Indicator Name'][0]]]

    WDI_df_marcap_test = WDI_df_marcap[WDI_df_marcap['Country Name'].isin([country])].reset_index(drop=True)
    WDI_df_marcap_test = WDI_df_marcap_test.rename(columns={"value": WDI_df_marcap_test['Indicator Name'][0], "variable": 'Year'})
    WDI_df_marcap_test = WDI_df_marcap_test[['Year', WDI_df_marcap_test['Indicator Name'][0]]]

    country_name = country
        
    WDI_df_test = pd.merge(WDI_df_marcap_test, WDI_df_gdp_test, how='left', on ='Year')
    
    print('######################################', country, '######################################')
    test = grangercausalitytests(WDI_df_test[[WDI_df_marcap_test.columns[1],WDI_df_gdp_test.columns[1]]], lagvar)

    ssr_based_chi2_marcap_by_gdp_cn_inter = pd.DataFrame([test[1][0]['ssr_chi2test'][0]] + [test[2][0]['ssr_chi2test'][0]] + [test[3][0]['ssr_chi2test'][0]] + [test[4][0]['ssr_chi2test'][0]] + [test[5][0]['ssr_chi2test'][0]]
                                                         + [test[6][0]['ssr_chi2test'][0]] + [test[7][0]['ssr_chi2test'][0]] + [test[8][0]['ssr_chi2test'][0]]
                                                         , columns=[country_name]).T
    ssr_based_chi2_marcap_by_gdp_cn = pd.concat([ssr_based_chi2_marcap_by_gdp_cn, ssr_based_chi2_marcap_by_gdp_cn_inter]) 

###################################### Brazil ######################################

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.6256  , p=0.2140  , df_denom=25, df_num=1
ssr based chi2 test:   chi2=1.8207  , p=0.1772  , df=1
likelihood ratio test: chi2=1.7639  , p=0.1841  , df=1
parameter F test:         F=1.6256  , p=0.2140  , df_denom=25, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6850  , p=0.5145  , df_denom=22, df_num=2
ssr based chi2 test:   chi2=1.6813  , p=0.4314  , df=2
likelihood ratio test: chi2=1.6310  , p=0.4424  , df=2
parameter F test:         F=0.6850  , p=0.5145  , df_denom=22, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.6659  , p=0.5833  , df_denom=19, df_num=3
ssr based chi2 test:   chi2=2.7338  , p=0.4345  , df=3
likelihood ratio test: chi2=2.5994  , p=0.4576  , df=3
parameter F test:         F=0.6659  , p=0.5833  , df_denom=19, df_num=3

Granger Causal

In [9]:
ssr_based_chi2_marcap_by_gdp_cn = ssr_based_chi2_marcap_by_gdp_cn.T

ssr_based_chi2_marcap_by_gdp_cn = ssr_based_chi2_marcap_by_gdp_cn.reset_index(drop=False)
ssr_based_chi2_marcap_by_gdp_cn['index'] = ssr_based_chi2_marcap_by_gdp_cn['index']+1
ssr_based_chi2_marcap_by_gdp_cn.rename(columns={'index': 'Degrees of Freedom'}, inplace=True)

chi2_critical_value_df = pd.DataFrame()

for deg_freedom in ssr_based_chi2_marcap_by_gdp_cn['Degrees of Freedom']:
    # set the significance level
    alpha = 0.05
    df = deg_freedom
    
    # calculate the critical value
    critical_value = chi2.ppf(1 - alpha, df)
    
    # buidling the df
    chi2_critical_value = pd.DataFrame({
        'Degrees of Freedom': [deg_freedom],
        'Chi-Square Critical Value at p-value = 0.05': [critical_value]
    })
    
    chi2_critical_value_df = pd.concat([chi2_critical_value_df, chi2_critical_value])

ssr_based_chi2_marcap_by_gdp_cn = pd.merge(ssr_based_chi2_marcap_by_gdp_cn, chi2_critical_value_df, how='left', on='Degrees of Freedom')

for col in ssr_based_chi2_marcap_by_gdp_cn.columns[1:-1]:
    ssr_based_chi2_marcap_by_gdp_cn[col] = ssr_based_chi2_marcap_by_gdp_cn[col] - ssr_based_chi2_marcap_by_gdp_cn['Chi-Square Critical Value at p-value = 0.05']

ssr_based_chi2_marcap_by_gdp_cn.insert(1,'Granger panel causality test (Dumitrescu and Hurlin (2012))', ssr_based_chi2_marcap_by_gdp_cn[ssr_based_chi2_marcap_by_gdp_cn.columns[1:-1]].T.mean())

ssr_based_chi2_marcap_by_gdp_cn = ssr_based_chi2_marcap_by_gdp_cn[['Degrees of Freedom',
                                                                   # 'Chi-Square Critical Value at p-value = 0.05', 
                                                                   'Brazil', 'China', 'Germany', 'Hong Kong SAR, China', 'India', 'Indonesia',
                                                                   'Iran, Islamic Rep.', 'Israel', 'Japan', 'Korea, Rep.', 'Luxembourg',
                                                                   'Malaysia', 'Russian Federation', 'Singapore', 'South Africa',
                                                                   'Turkiye', 'United States', 'Kuwait', 'Saudi Arabia',
                                                                   'Granger panel causality test (Dumitrescu and Hurlin (2012))']]

In [10]:
ssr_based_chi2_marcap_by_gdp_cn = ssr_based_chi2_marcap_by_gdp_cn.set_index("Degrees of Freedom").T
ssr_based_chi2_marcap_by_gdp_cn.to_excel(r"MarCap caused by GDP curr LCU.xlsx")
ssr_based_chi2_marcap_by_gdp_cn

Degrees of Freedom,1,2,3,4,5,6,7,8
Brazil,-2.020797,-4.310203,-5.080963,-6.193460,-5.580264,-10.292725,-12.001785,-9.360167
China,-3.579305,8.231093,11.270495,19.700714,17.769588,13.388366,40.334537,39.061372
Germany,-1.076410,-3.254785,-5.161115,-5.264475,-7.565606,-5.116404,-2.212672,30.130139
"Hong Kong SAR, China",-3.218432,5.487768,4.360314,2.080603,2.293568,3.519702,-0.304300,24.700096
India,-3.673637,-4.096277,18.700913,22.817948,15.139242,22.388524,16.478957,25.713144
Indonesia,-1.283790,0.736429,8.930625,24.856011,7.700860,21.845018,14.890024,14.241308
"Iran, Islamic Rep.",13.285620,14.670626,16.015618,27.664483,106.218165,1127.568753,273.691646,428.777279
Israel,0.574595,-0.816525,-3.174316,-2.399843,-3.400409,-1.661309,0.952021,-0.961864
Japan,-3.639812,-4.128798,-3.086208,14.130091,20.051024,10.380246,75.607942,258.502036
"Korea, Rep.",-11.562059,53.839066,-2.322658,-3.357056,0.746201,3.921843,15.530409,4.491067


## gdp is granger caused by marcap

In [11]:
ssr_based_chi2_gdp_by_marcap_cn = pd.DataFrame()

for country in WDI_df['Country Name'].unique():
    WDI_df_gdp_test = WDI_df_gdp[WDI_df_gdp['Country Name'].isin([country])].reset_index(drop=True)
    WDI_df_gdp_test = WDI_df_gdp_test.rename(columns={"value": WDI_df_gdp_test['Indicator Name'][0], "variable": 'Year'})

    WDI_df_gdp_test = WDI_df_gdp_test[['Year', WDI_df_gdp_test['Indicator Name'][0]]]

    WDI_df_marcap_test = WDI_df_marcap[WDI_df_marcap['Country Name'].isin([country])].reset_index(drop=True)
    WDI_df_marcap_test = WDI_df_marcap_test.rename(columns={"value": WDI_df_marcap_test['Indicator Name'][0], "variable": 'Year'})
    WDI_df_marcap_test = WDI_df_marcap_test[['Year', WDI_df_marcap_test['Indicator Name'][0]]]

    country_name = country
        
    WDI_df_test = pd.merge(WDI_df_marcap_test, WDI_df_gdp_test, how='left', on ='Year')

    print('######################################', country, '######################################')
    test = grangercausalitytests(WDI_df_test[[WDI_df_gdp_test.columns[1],WDI_df_marcap_test.columns[1]]], lagvar)

    ssr_based_chi2_gdp_by_marcap_cn_inter = pd.DataFrame([test[1][0]['ssr_chi2test'][0]] + [test[2][0]['ssr_chi2test'][0]] + [test[3][0]['ssr_chi2test'][0]] + [test[4][0]['ssr_chi2test'][0]] + [test[5][0]['ssr_chi2test'][0]]
                                                         + [test[6][0]['ssr_chi2test'][0]] + [test[7][0]['ssr_chi2test'][0]] + [test[8][0]['ssr_chi2test'][0]]
                                                         , columns=[country_name]).T
    ssr_based_chi2_gdp_by_marcap_cn = pd.concat([ssr_based_chi2_gdp_by_marcap_cn, ssr_based_chi2_gdp_by_marcap_cn_inter]) 

###################################### Brazil ######################################

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.9739  , p=0.1723  , df_denom=25, df_num=1
ssr based chi2 test:   chi2=2.2108  , p=0.1370  , df=1
likelihood ratio test: chi2=2.1279  , p=0.1446  , df=1
parameter F test:         F=1.9739  , p=0.1723  , df_denom=25, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.4842  , p=0.1065  , df_denom=22, df_num=2
ssr based chi2 test:   chi2=6.0977  , p=0.0474  , df=2
likelihood ratio test: chi2=5.4979  , p=0.0640  , df=2
parameter F test:         F=2.4842  , p=0.1065  , df_denom=22, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.3395  , p=0.0412  , df_denom=19, df_num=3
ssr based chi2 test:   chi2=13.7097 , p=0.0033  , df=3
likelihood ratio test: chi2=11.0110 , p=0.0117  , df=3
parameter F test:         F=3.3395  , p=0.0412  , df_denom=19, df_num=3

Granger Causal

In [12]:
ssr_based_chi2_gdp_by_marcap_cn = ssr_based_chi2_gdp_by_marcap_cn.T

ssr_based_chi2_gdp_by_marcap_cn = ssr_based_chi2_gdp_by_marcap_cn.reset_index(drop=False)
ssr_based_chi2_gdp_by_marcap_cn['index'] = ssr_based_chi2_gdp_by_marcap_cn['index']+1
ssr_based_chi2_gdp_by_marcap_cn.rename(columns={'index': 'Degrees of Freedom'}, inplace=True)

ssr_based_chi2_gdp_by_marcap_cn = pd.merge(ssr_based_chi2_gdp_by_marcap_cn, chi2_critical_value_df, how='left', on='Degrees of Freedom')

for col in ssr_based_chi2_gdp_by_marcap_cn.columns[1:-1]:
    ssr_based_chi2_gdp_by_marcap_cn[col] = ssr_based_chi2_gdp_by_marcap_cn[col] - ssr_based_chi2_gdp_by_marcap_cn['Chi-Square Critical Value at p-value = 0.05']

ssr_based_chi2_gdp_by_marcap_cn.insert(1,'Granger panel causality test (Dumitrescu and Hurlin (2012))', ssr_based_chi2_gdp_by_marcap_cn[ssr_based_chi2_gdp_by_marcap_cn.columns[1:-1]].T.mean())

ssr_based_chi2_gdp_by_marcap_cn = ssr_based_chi2_gdp_by_marcap_cn[['Degrees of Freedom',
                                                                   # 'Chi-Square Critical Value at p-value = 0.05', 
                                                                   'Brazil', 'China', 'Germany', 'Hong Kong SAR, China', 'India', 'Indonesia',
                                                                   'Iran, Islamic Rep.', 'Israel', 'Japan', 'Korea, Rep.', 'Luxembourg',
                                                                   'Malaysia', 'Russian Federation', 'Singapore', 'South Africa',
                                                                   'Turkiye', 'United States', 'Kuwait', 'Saudi Arabia',
                                                                   'Granger panel causality test (Dumitrescu and Hurlin (2012))']]

In [13]:
ssr_based_chi2_gdp_by_marcap_cn = ssr_based_chi2_gdp_by_marcap_cn.set_index("Degrees of Freedom").T
ssr_based_chi2_gdp_by_marcap_cn.to_excel(r"GDP caused by MarCap curr LCU.xlsx")
ssr_based_chi2_gdp_by_marcap_cn

Degrees of Freedom,1,2,3,4,5,6,7,8
Brazil,-1.630642,0.106204,5.894943,4.388323,22.086174,27.512209,50.117723,113.181970
China,2.208073,11.077717,1.403598,15.504268,17.006524,15.586031,92.062381,107.280992
Germany,-1.155879,-1.385291,-2.824384,-4.833846,-2.068657,-1.979211,-2.148089,22.615062
"Hong Kong SAR, China",1.649469,0.146155,2.733091,0.352723,2.015788,6.250352,5.182593,3.004493
India,2.991255,5.759722,-1.150464,-3.155277,-3.562916,-0.736976,36.791951,113.165142
Indonesia,-1.971940,-4.018455,-6.204588,-3.706129,-4.703996,0.031535,14.219340,70.770292
"Iran, Islamic Rep.",18.445655,33.822689,24.290857,23.059138,34.253837,1503.365604,881.758941,1305.889873
Israel,-3.648563,-4.781164,-3.450999,-5.233753,-4.397053,-5.837016,-7.105208,-6.149344
Japan,0.631594,4.321292,5.273619,13.146392,9.091985,5.045413,7.737950,49.579118
"Korea, Rep.",9.263134,-2.722785,-4.551430,-5.118917,20.778161,43.153854,42.877514,48.110612


# Current US$

In [14]:
WDI_df_marcap = WDI_df[WDI_df['Indicator Code'].isin(['CM.MKT.LCAP.CD'])].reset_index(drop=True)
WDI_df_gdp = WDI_df[WDI_df['Indicator Code'].isin(['NY.GDP.MKTP.CD'])].reset_index(drop=True)

lagvar = 8

## marcap is granger caused by gdp

In [15]:
ssr_based_chi2_marcap_by_gdp_cd = pd.DataFrame()

for country in WDI_df['Country Name'].unique():
    WDI_df_gdp_test = WDI_df_gdp[WDI_df_gdp['Country Name'].isin([country])].reset_index(drop=True)
    WDI_df_gdp_test = WDI_df_gdp_test.rename(columns={"value": WDI_df_gdp_test['Indicator Name'][0], "variable": 'Year'})

    WDI_df_gdp_test = WDI_df_gdp_test[['Year', WDI_df_gdp_test['Indicator Name'][0]]]

    WDI_df_marcap_test = WDI_df_marcap[WDI_df_marcap['Country Name'].isin([country])].reset_index(drop=True)
    WDI_df_marcap_test = WDI_df_marcap_test.rename(columns={"value": WDI_df_marcap_test['Indicator Name'][0], "variable": 'Year'})
    WDI_df_marcap_test = WDI_df_marcap_test[['Year', WDI_df_marcap_test['Indicator Name'][0]]]

    country_name = country
        
    WDI_df_test = pd.merge(WDI_df_marcap_test, WDI_df_gdp_test, how='left', on ='Year')
    
    print('######################################', country, '######################################')
    test = grangercausalitytests(WDI_df_test[[WDI_df_marcap_test.columns[1],WDI_df_gdp_test.columns[1]]], lagvar)

    ssr_based_chi2_marcap_by_gdp_cd_inter = pd.DataFrame([test[1][0]['ssr_chi2test'][0]] + [test[2][0]['ssr_chi2test'][0]] + [test[3][0]['ssr_chi2test'][0]] + [test[4][0]['ssr_chi2test'][0]] + [test[5][0]['ssr_chi2test'][0]]
                                                         + [test[6][0]['ssr_chi2test'][0]] + [test[7][0]['ssr_chi2test'][0]] + [test[8][0]['ssr_chi2test'][0]]
                                                         , columns=[country_name]).T
    ssr_based_chi2_marcap_by_gdp_cd = pd.concat([ssr_based_chi2_marcap_by_gdp_cd, ssr_based_chi2_marcap_by_gdp_cd_inter]) 

###################################### Brazil ######################################

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0006  , p=0.9813  , df_denom=25, df_num=1
ssr based chi2 test:   chi2=0.0006  , p=0.9800  , df=1
likelihood ratio test: chi2=0.0006  , p=0.9800  , df=1
parameter F test:         F=0.0006  , p=0.9813  , df_denom=25, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.2347  , p=0.7927  , df_denom=22, df_num=2
ssr based chi2 test:   chi2=0.5762  , p=0.7497  , df=2
likelihood ratio test: chi2=0.5701  , p=0.7520  , df=2
parameter F test:         F=0.2347  , p=0.7927  , df_denom=22, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.0487  , p=0.9853  , df_denom=19, df_num=3
ssr based chi2 test:   chi2=0.2000  , p=0.9776  , df=3
likelihood ratio test: chi2=0.1992  , p=0.9777  , df=3
parameter F test:         F=0.0487  , p=0.9853  , df_denom=19, df_num=3

Granger Causal

In [16]:
ssr_based_chi2_marcap_by_gdp_cd = ssr_based_chi2_marcap_by_gdp_cd.T

ssr_based_chi2_marcap_by_gdp_cd = ssr_based_chi2_marcap_by_gdp_cd.reset_index(drop=False)
ssr_based_chi2_marcap_by_gdp_cd['index'] = ssr_based_chi2_marcap_by_gdp_cd['index']+1
ssr_based_chi2_marcap_by_gdp_cd.rename(columns={'index': 'Degrees of Freedom'}, inplace=True)

chi2_critical_value_df = pd.DataFrame()

for deg_freedom in ssr_based_chi2_marcap_by_gdp_cd['Degrees of Freedom']:
    # set the significance level
    alpha = 0.05
    df = deg_freedom
    
    # calculate the critical value
    critical_value = chi2.ppf(1 - alpha, df)
    
    # buidling the df
    chi2_critical_value = pd.DataFrame({
        'Degrees of Freedom': [deg_freedom],
        'Chi-Square Critical Value at p-value = 0.05': [critical_value]
    })
    
    chi2_critical_value_df = pd.concat([chi2_critical_value_df, chi2_critical_value])

ssr_based_chi2_marcap_by_gdp_cd = pd.merge(ssr_based_chi2_marcap_by_gdp_cd, chi2_critical_value_df, how='left', on='Degrees of Freedom')

for col in ssr_based_chi2_marcap_by_gdp_cd.columns[1:-1]:
    ssr_based_chi2_marcap_by_gdp_cd[col] = ssr_based_chi2_marcap_by_gdp_cd[col] - ssr_based_chi2_marcap_by_gdp_cd['Chi-Square Critical Value at p-value = 0.05']

ssr_based_chi2_marcap_by_gdp_cd.insert(1,'Granger panel causality test (Dumitrescu and Hurlin (2012))', ssr_based_chi2_marcap_by_gdp_cd[ssr_based_chi2_marcap_by_gdp_cd.columns[1:-1]].T.mean())

ssr_based_chi2_marcap_by_gdp_cd = ssr_based_chi2_marcap_by_gdp_cd[['Degrees of Freedom',
                                                                   # 'Chi-Square Critical Value at p-value = 0.05', 
                                                                   'Brazil', 'China', 'Germany', 'Hong Kong SAR, China', 'India', 'Indonesia',
                                                                   'Iran, Islamic Rep.', 'Israel', 'Japan', 'Korea, Rep.', 'Luxembourg',
                                                                   'Malaysia', 'Russian Federation', 'Singapore', 'South Africa',
                                                                   'Turkiye', 'United States', 'Kuwait', 'Saudi Arabia',
                                                                   'Granger panel causality test (Dumitrescu and Hurlin (2012))']]

In [17]:
ssr_based_chi2_marcap_by_gdp_cd = ssr_based_chi2_marcap_by_gdp_cd.set_index("Degrees of Freedom").T
ssr_based_chi2_marcap_by_gdp_cd.to_excel(r"MarCap caused by GDP curr US.xlsx")
ssr_based_chi2_marcap_by_gdp_cd

Degrees of Freedom,1,2,3,4,5,6,7,8
Brazil,-3.840830,-5.415274,-7.614718,-5.780912,-6.942655,-7.343959,-8.299718,121.693539
China,-2.060230,12.531541,10.921142,15.671954,17.800699,17.684302,29.220925,29.515534
Germany,-1.741274,-5.406597,-5.685677,-6.613801,-0.338831,-7.704690,-5.614346,4.674026
"Hong Kong SAR, China",-3.097308,4.713315,3.858556,1.460025,1.380230,3.287840,-0.494379,32.144859
India,-2.262101,-5.002092,-5.548430,-3.896194,-6.844260,-7.807202,-8.365291,-7.821163
Indonesia,-3.404166,-2.955367,-0.817708,3.113259,0.251771,16.130900,9.925624,19.775400
"Iran, Islamic Rep.",-3.515638,-1.195396,-0.813879,10.956775,10.572969,15.338774,14.046415,59.191158
Israel,1.707122,-0.742855,-2.649419,-3.513795,21.047635,21.391066,38.153885,41.910433
Japan,-3.840916,-2.744294,-6.467629,-8.843278,-8.215632,-10.517016,4.435344,-1.502816
"Korea, Rep.",5.210039,-5.221565,-6.679410,-7.412019,-7.474452,-4.605344,0.746661,-0.819487


## gdp is granger caused by marcap

In [18]:
ssr_based_chi2_gdp_by_marcap_cd = pd.DataFrame()

for country in WDI_df['Country Name'].unique():
    WDI_df_gdp_test = WDI_df_gdp[WDI_df_gdp['Country Name'].isin([country])].reset_index(drop=True)
    WDI_df_gdp_test = WDI_df_gdp_test.rename(columns={"value": WDI_df_gdp_test['Indicator Name'][0], "variable": 'Year'})

    WDI_df_gdp_test = WDI_df_gdp_test[['Year', WDI_df_gdp_test['Indicator Name'][0]]]

    WDI_df_marcap_test = WDI_df_marcap[WDI_df_marcap['Country Name'].isin([country])].reset_index(drop=True)
    WDI_df_marcap_test = WDI_df_marcap_test.rename(columns={"value": WDI_df_marcap_test['Indicator Name'][0], "variable": 'Year'})
    WDI_df_marcap_test = WDI_df_marcap_test[['Year', WDI_df_marcap_test['Indicator Name'][0]]]

    country_name = country
        
    WDI_df_test = pd.merge(WDI_df_marcap_test, WDI_df_gdp_test, how='left', on ='Year')

    print('######################################', country, '######################################')
    test = grangercausalitytests(WDI_df_test[[WDI_df_gdp_test.columns[1],WDI_df_marcap_test.columns[1]]], lagvar)

    ssr_based_chi2_gdp_by_marcap_cd_inter = pd.DataFrame([test[1][0]['ssr_chi2test'][0]] + [test[2][0]['ssr_chi2test'][0]] + [test[3][0]['ssr_chi2test'][0]] + [test[4][0]['ssr_chi2test'][0]] + [test[5][0]['ssr_chi2test'][0]]
                                                         + [test[6][0]['ssr_chi2test'][0]] + [test[7][0]['ssr_chi2test'][0]] + [test[8][0]['ssr_chi2test'][0]]
                                                         , columns=[country_name]).T
    ssr_based_chi2_gdp_by_marcap_cd = pd.concat([ssr_based_chi2_gdp_by_marcap_cd, ssr_based_chi2_gdp_by_marcap_cd_inter]) 

###################################### Brazil ######################################

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=5.8399  , p=0.0233  , df_denom=25, df_num=1
ssr based chi2 test:   chi2=6.5407  , p=0.0105  , df=1
likelihood ratio test: chi2=5.8782  , p=0.0153  , df=1
parameter F test:         F=5.8399  , p=0.0233  , df_denom=25, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.6233  , p=0.0951  , df_denom=22, df_num=2
ssr based chi2 test:   chi2=6.4389  , p=0.0400  , df=2
likelihood ratio test: chi2=5.7749  , p=0.0557  , df=2
parameter F test:         F=2.6233  , p=0.0951  , df_denom=22, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.5045  , p=0.2455  , df_denom=19, df_num=3
ssr based chi2 test:   chi2=6.1765  , p=0.1033  , df=3
likelihood ratio test: chi2=5.5416  , p=0.1362  , df=3
parameter F test:         F=1.5045  , p=0.2455  , df_denom=19, df_num=3

Granger Causal

In [19]:
ssr_based_chi2_gdp_by_marcap_cd = ssr_based_chi2_gdp_by_marcap_cd.T

ssr_based_chi2_gdp_by_marcap_cd = ssr_based_chi2_gdp_by_marcap_cd.reset_index(drop=False)
ssr_based_chi2_gdp_by_marcap_cd['index'] = ssr_based_chi2_gdp_by_marcap_cd['index']+1
ssr_based_chi2_gdp_by_marcap_cd.rename(columns={'index': 'Degrees of Freedom'}, inplace=True)

ssr_based_chi2_gdp_by_marcap_cd = pd.merge(ssr_based_chi2_gdp_by_marcap_cd, chi2_critical_value_df, how='left', on='Degrees of Freedom')

for col in ssr_based_chi2_gdp_by_marcap_cd.columns[1:-1]:
    ssr_based_chi2_gdp_by_marcap_cd[col] = ssr_based_chi2_gdp_by_marcap_cd[col] - ssr_based_chi2_gdp_by_marcap_cd['Chi-Square Critical Value at p-value = 0.05']

ssr_based_chi2_gdp_by_marcap_cd.insert(1,'Granger panel causality test (Dumitrescu and Hurlin (2012))', ssr_based_chi2_gdp_by_marcap_cd[ssr_based_chi2_gdp_by_marcap_cd.columns[1:-1]].T.mean())

ssr_based_chi2_gdp_by_marcap_cd = ssr_based_chi2_gdp_by_marcap_cd[['Degrees of Freedom',
                                                                   # 'Chi-Square Critical Value at p-value = 0.05',
                                                                   'Brazil', 'China', 'Germany', 'Hong Kong SAR, China', 'India', 'Indonesia',
                                                                   'Iran, Islamic Rep.', 'Israel', 'Japan', 'Korea, Rep.', 'Luxembourg',
                                                                   'Malaysia', 'Russian Federation', 'Singapore', 'South Africa',
                                                                   'Turkiye', 'United States', 'Kuwait', 'Saudi Arabia',
                                                                   'Granger panel causality test (Dumitrescu and Hurlin (2012))']]

In [20]:
ssr_based_chi2_gdp_by_marcap_cd = ssr_based_chi2_gdp_by_marcap_cd.set_index("Degrees of Freedom").T
ssr_based_chi2_gdp_by_marcap_cd.to_excel(r"GDP caused by MarCap curr US.xlsx")
ssr_based_chi2_gdp_by_marcap_cd

Degrees of Freedom,1,2,3,4,5,6,7,8
Brazil,2.699283,0.447463,-1.638270,-2.647383,13.843226,17.617116,67.457953,273.641322
China,1.941069,1.758879,-6.680922,-6.136549,-3.684215,28.216383,51.396230,1966.406768
Germany,1.542048,0.797343,9.272157,20.482605,19.686603,23.226730,75.859691,167.723707
"Hong Kong SAR, China",2.603256,0.914345,3.874456,1.385433,3.538104,8.896607,9.153981,7.243677
India,-2.666955,-4.381325,-5.685177,-7.869257,-1.657117,14.682695,9.033807,28.418573
Indonesia,0.643595,-1.649688,-0.960101,5.517574,9.951192,25.703466,54.525572,54.787289
"Iran, Islamic Rep.",1.376211,-0.451827,1.833629,9.649240,6.063914,36.269988,58.680915,68.249054
Israel,2.958249,2.993416,-1.407168,18.452948,27.326050,28.078657,55.509549,106.909822
Japan,-1.179521,0.903643,0.533283,0.334271,-0.232507,1.010558,6.925515,20.882732
"Korea, Rep.",11.532975,5.233163,5.925680,5.288809,38.226432,50.129698,56.106809,47.084175
